# CSV to SQL

In [13]:
import pandas as pd
import numpy as np
import sys

# Function Definition

In [14]:
def DataframeToCsv(dataframe, fileName, tableName, columns):
    numRow = len(dataframe.index)
    numCol = len(dataframe.columns)
    
    with open(fileName, 'w') as f:
        for row in range(0, numRow):
            print("INSERT INTO " + tableName + " " + columns + " VALUES (", file=f, end="")
            for column in range(0, numCol):
                print("NULL" 
                      if ((dataframe.iat[row, column] == "'nan'")
                          or (dataframe.iat[row, column] == "—")
                          or (dataframe.iat[row, column] == "'(none)'"))
                      else dataframe.iat[row, column].replace("\n", "\\n"), file=f, end="")
                if (column != numCol-1):
                    print(", ", file=f, end="")
                else:
                    print(");", file=f)

# Country

In [15]:
# Reads all CSVs
country = pd.read_csv("raw-data/List_of_countries_and_dependencies_by_population_density.csv")
capital = pd.read_csv("raw-data/List_of_national_capitals_1.csv")
continents = pd.read_csv("raw-data/List_of_countries_by_GDP_(nominal)_1.csv") # Get the continent of the country

# Treats CSV
continents = continents[['Country/Territory', 'UN Region']].copy()
capital.drop("Notes", axis=1, inplace=True)
country.drop("Source and date", axis=1, inplace=True)
country.drop("Area.1", axis=1, inplace=True)
country.drop("Population density.1", axis=1, inplace=True)
country.drop(0, axis=0, inplace=True)

country = country.join(capital.set_index("Country/Territory"), on="Country, territory or dependency")
country = country.join(continents.set_index("Country/Territory"), on="Country, territory or dependency")

# Format strings
str_cols = country.select_dtypes(['object']).columns
country[str_cols] = country[str_cols].replace(",", "", regex=True).replace("'", "''", regex=True)
country.update(country[[
    "Country, territory or dependency",
    "City/Town",
    "UN Region"
]].applymap("'{}'".format))

country = country.sort_values("Country, territory or dependency")

#country.head()

# GDP

In [16]:
gdp = pd.read_csv("raw-data/List_of_countries_by_GDP_(nominal)_1.csv")

# Treats CSV
gdp.drop("UN Region", axis=1, inplace=True)
gdp.drop("IMF.1", axis=1, inplace=True)
gdp.drop("World Bank.1", axis=1, inplace=True)
gdp.drop("United Nations.1", axis=1, inplace=True)
gdp.drop(0, axis=0, inplace=True)
gdp.drop(1, axis=0, inplace=True)

gdp.rename({"IMF" : "2022", "World Bank" : "2021", "United Nations" : "2020"}, axis=1, inplace=True)
gdp.drop("2021", axis=1, inplace=True)
gdp.drop("2020", axis=1, inplace=True)
str_cols = gdp.select_dtypes(["object"]).columns
gdp[str_cols] = gdp[str_cols].replace(",", "", regex=True)

gdp.update(gdp[["Country/Territory"]].applymap("'{}'".format))
gdp = gdp.sort_values("Country/Territory")

#gdp.head()

# Head of State

In [17]:
headState = pd.read_csv("raw-data/List_of_current_heads_of_state_and_government_1.csv")
headState2 = pd.read_csv("raw-data/List_of_current_heads_of_state_and_government_2.csv")
headState3 = pd.read_csv("raw-data/List_of_current_heads_of_state_and_government_3.csv")

# Treats CSV
headState2.drop("Associated with", axis=1, inplace=True)
headState3.drop("Also claimed by", axis=1, inplace=True)
headState = headState.append(headState2)
headState = headState.append(headState3)
headState.drop("Head of government", axis=1, inplace=True)

str_cols = headState.select_dtypes(["object"]).columns
headState[str_cols] = headState[str_cols].replace("'", "''", regex=True).applymap("'{}'".format)

headState = headState.sort_values("State")

headState.head()

/tmp/ipykernel_43338/1527354819.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  headState = headState.append(headState2)
/tmp/ipykernel_43338/1527354819.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  headState = headState.append(headState3)


,State,Head of state
0,'Abkhazia','President – Aslan Bzhania'
0,'Afghanistan','Supreme Leader – Hibatullah Akhundzada'
1,'Albania','President – Bajram Begaj'
2,'Algeria','President – Abdelmadjid Tebboune'
3,'Andorra','Episcopal Co-Prince – Joan Enric Vives i Sicí...


# Language

In [18]:
language = pd.read_csv("raw-data/List_of_official_languages_by_country_and_territory_1.csv")

# Treats CSV
language = language[["Country/Region", "Official language"]].copy()

str_cols = language.select_dtypes(["object"]).columns
language[str_cols] = language[str_cols].replace("'", "''", regex=True).applymap("'{}'".format)

#language.head()

# Religion

In [19]:
religion = pd.read_csv("raw-data/Religions_by_country.csv")

# Treats CSV
religion.drop("Population", axis=1, inplace=True)
religion.drop("Christian.1", axis=1, inplace=True)
religion.drop("Muslim.1", axis=1, inplace=True)
religion.drop("Irreligion.1", axis=1, inplace=True)
religion.drop("Hindu.1", axis=1, inplace=True)
religion.drop("Buddhist.1", axis=1, inplace=True)
religion.drop("Folk religion.1", axis=1, inplace=True)
religion.drop("Other religion.1", axis=1, inplace=True)
religion.drop("Jewish.1", axis=1, inplace=True)

str_cols = religion.select_dtypes(["object"]).columns
religion[str_cols] = religion[str_cols].replace(",", "", regex=True)
religion.update(religion[["Country"]].applymap("'{}'".format))

#religion.head()

# Currency

In [20]:
currency = pd.read_csv("raw-data/List_of_circulating_currencies_1.csv")

# Treats CSV
str_cols = currency.select_dtypes(['object']).columns
currency[str_cols] = currency[str_cols].replace("'", "''", regex=True)
currency.update(currency[[
   "State or territory",
    "Currency",
    "Symbol or\nAbbrev.",
    "ISO code", 
    "Fractional\nunit"
]].applymap("'{}'".format))

#currency.head()

# Funcion Calls

In [21]:
DataframeToCsv(country, "cooked-data/country.sql", "country", "()")
DataframeToCsv(gdp, "cooked-data/gdp.sql", "gdp", "()")
DataframeToCsv(headState, "cooked-data/head_of_state.sql", "head_of_state", "()")
DataframeToCsv(language, "cooked-data/language.sql", "language", "()")
DataframeToCsv(religion, "cooked-data/religion.sql", "religion", "()")
DataframeToCsv(currency, "cooked-data/currency.sql", "currency", "()")